In [2]:
import pandas as pd

from google.colab import files
uploaded = files.upload()  # "토마토_유전형.CSV", "토마토_표현형.CSV"


Saving 토마토_표현형.CSV to 토마토_표현형.CSV


In [3]:
import pandas as pd
import numpy as np

# ✅ 데이터 불러오기
geno_df = pd.read_csv("토마토_유전형.CSV")
pheno_df = pd.read_csv("토마토_표현형.CSV", encoding='cp949')  # 한글 인코딩 고려

In [4]:
# 데이터 미리보기
print("유전형 데이터:")
print(geno_df.head())
print("\n표현형 데이터:")
print(pheno_df.head())


유전형 데이터:
        SNP ID  Chr  Position (bp) TC1_175 TC1_187 TC1_001 TC1_009 TC1_016  \
0  AX.95808842    1          20288      CC      TT      TT      TT      TT   
1  AX.95772159    1          62862      TT      CC      TT      TT      CC   
2  AX.95771895    1          65279      TT      TT      TT      TT      TT   
3  AX.95770515    1          65409      GG      GG      GG      GG      GG   
4  AX.95792562    1          65869      CC      CC      CC      CC      CC   

  TC1_024 TC1_032  ... TC1_103 TC1_112 TC1_120 TC1_128 TC1_136 TC1_144  \
0      CC      CC  ...      TT      CC      TT      TT      CC      TT   
1      TT      TT  ...      CC      TT      CC      CC      TT      TT   
2      TT      TT  ...      TT      TT      CC      CC      TT      TT   
3      GG      GG  ...      GG      GG      AA      AA      GG      GG   
4      CC      CC  ...      CC      CC      TT      TT      CC      CC   

  TC1_152 TC1_160 TC1_168 TC1_109  
0      TT      TT      TT      TT  
1    

In [10]:
# ✅ .map 파일 생성
map_df = geno_df[["Chr", "SNP ID", "Position (bp)"]].copy()
map_df.insert(2, "GenDist", 0)
map_df.columns = ["Chr", "SNP", "GenDist", "BP"]
map_df.to_csv("geno.map", sep="\t", index=False, header=False)

print("✅ .map 파일 생성 완료!")


✅ .map 파일 생성 완료!


In [11]:
# ✅ .ped 파일 생성
import pandas as pd
import numpy as np

# 🔹 파일 경로
geno_path = "토마토_유전형.CSV"
pheno_path = "토마토_표현형.CSV"

# 1. 파일 불러오기
geno_raw = pd.read_csv(geno_path, index_col=0)  # SNP가 열로 있음
pheno_df = pd.read_csv(pheno_path, index_col=0, encoding='cp949')  # SampleID 기준

# 2. 유전형 전처리: "Chr", "Position (bp)" 제거 후 전치
geno_T = geno_raw.drop(columns=["Chr", "Position (bp)"]).T.replace("NN", np.nan)

# 3. SNP별 유전형 문자 → 숫자 변환 필요 없음 (PLINK는 문자로도 가능), 다만 'NN'은 결측치로 처리 → '0 0'
geno_T = geno_T.fillna("00")  # 결측치는 00으로 (0 0)

# 4. 표현형 병합
geno_T["SampleID"] = geno_T.index
pheno_df["SampleID"] = pheno_df.index
merged = pd.merge(pheno_df, geno_T, on="SampleID")

# 5. .ped 고정 컬럼 구성 (FID, IID, PID, MID, Sex, Phenotype)
n = len(merged)
ped_fixed = pd.DataFrame({
    "FID": merged["SampleID"],
    "IID": merged["SampleID"],
    "PID": 0,
    "MID": 0,
    "Sex": 1,
    "Phenotype": merged["당도 (%)"]  # 예시로 당도(%) 사용
})

# 6. 유전형 데이터 변환: AA → A A 형식으로 나누기
geno_data = merged.drop(columns=pheno_df.columns.tolist())  # SNP만 추출
geno_split = geno_data.applymap(lambda x: f"{x[0]} {x[1]}" if len(x) == 2 else "0 0")

# 7. 최종 .ped 파일 구성
geno_expanded = geno_split.apply(lambda row: " ".join(row), axis=1).str.split(" ", expand=True)
ped_final = pd.concat([ped_fixed.reset_index(drop=True), geno_expanded], axis=1)
ped_final.to_csv("output.ped", sep=" ", index=False, header=False)

print("✅ .ped 파일만 생성 완료!")


<ipython-input-11-728792d26420>:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  geno_split = geno_data.applymap(lambda x: f"{x[0]} {x[1]}" if len(x) == 2 else "0 0")


✅ .ped 파일만 생성 완료!


In [12]:
from google.colab import files

# .ped, .map, .pheno 파일 업로드
uploaded = files.upload()  # output.ped, geno.map, 과중.pheno 등 업로드


Saving 당도.pheno to 당도.pheno
Saving 과피두께.pheno to 과피두께.pheno
Saving 과폭.pheno to 과폭.pheno
Saving 과중.pheno to 과중.pheno
Saving 과장.pheno to 과장.pheno
Saving 과실경도.pheno to 과실경도.pheno


In [13]:
# 최신 PLINK 설치
!wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20210606.zip
!unzip plink_linux_x86_64_20210606.zip
!chmod +x plink


--2025-05-16 06:39:39--  http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20210606.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.98.86, 16.182.34.152, 3.5.24.47, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.98.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8917076 (8.5M) [application/zip]
Saving to: ‘plink_linux_x86_64_20210606.zip’

plink_linux_x86_64_ 100%[===================>]   8.50M  --.-KB/s    in 0.1s    

2025-05-16 06:39:40 (77.7 MB/s) - ‘plink_linux_x86_64_20210606.zip’ saved [8917076/8917076]

Archive:  plink_linux_x86_64_20210606.zip
  inflating: plink                   
  inflating: LICENSE                 
  inflating: toy.ped                 
  inflating: toy.map                 
  inflating: prettify                


In [14]:
!./plink --file output --make-bed --out tomato_bin


PLINK v1.90b6.24 64-bit (6 Jun 2021)           www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tomato_bin.log.
Options in effect:
  --file output
  --make-bed
  --out tomato_bin

12977 MB RAM detected; reserving 6488 MB for main workspace.
Error: Failed to open output.map.


In [15]:
import os

# 업로드된 파일명 확인 후 강제 변경
os.rename("output.ped", "output.ped")
os.rename("geno.map", "output.map")  # PLINK는 .map 확장자 요구

# 확인
!ls -l output.*


-rw-r--r-- 1 root root  1266799 May 16 06:35 output.map
-rw-r--r-- 1 root root 39337344 May 16 06:37 output.ped


In [16]:
!./plink --file output --make-bed --out tomato_bin


PLINK v1.90b6.24 64-bit (6 Jun 2021)           www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tomato_bin.log.
Options in effect:
  --file output
  --make-bed
  --out tomato_bin

12977 MB RAM detected; reserving 6488 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (51214 variants, 192 people).
--file: tomato_bin-temporary.bed + tomato_bin-temporary.bim +
tomato_bin-temporary.fam written.
51214 variants loaded from .bim file.
192 people (192 males, 0 females) loaded from .fam.
192 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 192 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%

In [17]:
# plink로 gwas 실행해서 assoclinear 파일 생성
# 표현형 파일 리스트 (업로드한 파일 이름 기준)
pheno_files = [
    "과중.pheno",
    "과장.pheno",
    "과폭.pheno",
    "과피두께.pheno",
    "과실경도.pheno",
    "당도.pheno"
]

# 결과를 담을 딕셔너리
top_snps_dict = {}


import os

for pheno in pheno_files:
    trait = pheno.replace(".pheno", "")
    out_prefix = f"gwas_{trait}"

    # GWAS 실행 (linear regression)
    !./plink --bfile tomato_bin --pheno {pheno} --linear --out {out_prefix}


PLINK v1.90b6.24 64-bit (6 Jun 2021)           www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to gwas_과중.log.
Options in effect:
  --bfile tomato_bin
  --linear
  --out gwas_과중
  --pheno 과중.pheno

12977 MB RAM detected; reserving 6488 MB for main workspace.
51214 variants loaded from .bim file.
192 people (192 males, 0 females) loaded from .fam.
192 phenotype values present after --pheno.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 192 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%64%65%6

In [24]:
# 상위 snp 추출
import pandas as pd
import os

# 🔧 상위 SNP 개수 설정
top_n = 2000  # 원하는 개수로 수정 (예: 50, 100 등)

# 표현형 파일 리스트
pheno_files = [
    "과중.pheno", "과장.pheno", "과폭.pheno", "과피두께.pheno", "과실경도.pheno", "당도.pheno"
]

# 결과를 누적 저장할 리스트
all_top_snps = []

for pheno in pheno_files:
    trait = pheno.replace(".pheno", "")
    result_file = f"gwas_{trait}.assoc.linear"

    if os.path.exists(result_file):
        df = pd.read_csv(result_file, delim_whitespace=True)

        # TEST 열이 있으면 ADD만 선택, 없으면 전체 사용
        if "TEST" in df.columns:
            df = df[df["TEST"] == "ADD"]

        # 상위 n개 SNP 추출
        top_snps = df.sort_values("P").head(top_n).copy()
        top_snps["Trait"] = trait  # 어떤 형질의 결과인지 표시

        # 필요한 열만 선택
        selected = top_snps[["Trait", "SNP", "P", "BETA"]]
        all_top_snps.append(selected)
    else:
        print(f"❌ GWAS 결과 파일 없음: {result_file}")

# 통합 DataFrame 생성
top_snps_df = pd.concat(all_top_snps, ignore_index=True)

# CSV 저장
output_file = f"GWAS_top{top_n}_snps_all_traits.csv"
top_snps_df.to_csv(output_file, index=False)
print(f"✅ 저장 완료: {output_file}")


<ipython-input-24-fb607a5ca064>:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(result_file, delim_whitespace=True)
<ipython-input-24-fb607a5ca064>:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(result_file, delim_whitespace=True)
<ipython-input-24-fb607a5ca064>:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(result_file, delim_whitespace=True)
<ipython-input-24-fb607a5ca064>:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(result_file, delim_whitespace=True)
<ipython-input-24-fb607a5ca064>:20: FutureWarning: The 'delim_whites

✅ 저장 완료: GWAS_top2000_snps_all_traits.csv


<ipython-input-24-fb607a5ca064>:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(result_file, delim_whitespace=True)
